# Constructing a simple model for tidal displacements due to density perturbations

We wish to build a forward model that predicts surface displacements (for example, GPS tidal amplitudes) corresponding to density perturbations within the mantle. Since the full set of equations is quite complicated, we will simplify the equations to the radial dimension (depth dimension).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

import jax
import jax.numpy as jnp

In [ ]:
# Define parameters
Re = 6371.0e3  # Earth's radius in meters
Rc = 3486.0e3 # Radius of Earth's core in meters
R = Re - Rc # radius from Earth's core to the surface

# Construct the vector of radius values where r = 0 corresponds
# to the CMB, and r = R corresponds to the Earth's surface
dR = 5.0e3
rvec = np.arange(dR, R + 0.1*dR, dR)
print(f"Number of radius values: {rvec.size}")

## Define the equations of motion for radial displacement due to tidal forcing

We construct an ordinary differential equation (ODE) of the form:
$$
\frac{d^2 u}{dr^2} = -\frac{2}{r}\frac{du}{dr} + \left(\frac{6}{r^2} - \frac{\rho \omega^2}{\lambda + 2\mu}\right) u + \frac{1}{\lambda + 2\mu}\frac{d\Phi}{dr},
$$
where $u = u(r)$ is the displacement as a function of $r$, $\lambda$ and $\mu$ are the Lamé parameters, $\rho$ is the reference density, $\Phi$ is the tidal potential field, and $\omega$ is the _eigenfrequency_. We will use this equation to find the __normal modes__ of oscillation in the radial dimension.

Note, in this example, we will construct linear profiles for the variations in Vp, Vs, and density. We will also assume a simplified form of the tidal potential field, which will generally depend on the tidal model used.

In [ ]:
# Define the Vp profile as a function of radius
def Vp(r):
    Vp0 = 14.0e3
    Vp1 = 8.0e3
    return Vp0 + (Vp1 - Vp0) * r / R

# Define the Vs profile as a function of radius
def Vs(r):
    Vs0 = 7.0e3
    Vs1 = 4.0e3
    return Vs0 + (Vs1 - Vs0) * r / R

# Define the density profile as a function of radius
def rho(r):
    rho0 = 5200.0
    rho1 = 3300.0
    return rho0 + (rho1 - rho0) * r / R

# Define the sum of the Lame parameters (λ + 2μ) as a function of radius
# Note: since λ = ρ * Vp(r)**2 - 2 * μ, we simply have to compute ρ * Vp(r)**2
def Lame(r):
    return rho(r) * Vp(r)**2

# Define the tidal potential (simplified form)
def tidal_potential(r):
    # Assuming a form for the tidal potential
    # This needs to be adjusted based on the specific tidal model used
    A = 1.0e-8  # Amplitude of tidal potential (example value)
    return A * r**2  # Simplified quadratic form for demonstration

# Define the ODE system
@jax.jit
def _ode(r, y, omega):
    # Unpack the state vector
    u, v = y
    dudr = v

    # Compute elastic quantities
    ρ = rho(r)
    L = Lame(r)
    μ = ρ * Vs(r)**2

    # Tidal force is the gradient of the tidal potential
    f = jax.grad(tidal_potential)(r)

    # Evaluate ODE
    dvdr = (-2 / r) * v + (6 / r**2 - ρ * omega**2 / L) * u + f / L
    
    return jnp.array([jnp.squeeze(dudr), jnp.squeeze(dvdr)])

# Define the Jacobian of the ODE system outputs with respect to its inputs
_jac_ode = jax.jit(jax.jacfwd(_ode, argnums=1))

# Wrap the ODE functions to return Numpy arrays instead of JAX arrays
# to be compatible with LSODA
def radial_ode(r, y, omega):
    return np.array(_ode(r, y, omega))

def jac_radial_ode(r, y, omega):
    return np.array(_jac_ode(r, y, omega))

# Evaluate the ODE for a test point in order to jit-compile the functions
temp = radial_ode(0.1*R, [0.1, 0.0], 1.0e-3)
print('Test ODE output:', temp)

fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(12, 4))
ax1.plot(rvec, Vp(rvec))
ax2.plot(rvec, Vs(rvec))
ax3.plot(rvec, rho(rvec))
ax1.set_title('Vp (m/s)')
ax2.set_title('Vs (m/s)')
ax3.set_title(r'Density (kg/m$^3$)')
for ax in (ax1, ax2, ax3):
    ax.set_xlabel('Distance from CMB (m)')
plt.tight_layout()

## Solving for the normal modes

We can solve the ODE for any frequency $\omega$. However, only the solutions that satisfy the __zero-stress surface boundary condition__ are considered to be valid normal modes. We therefore need to define a function that computes the stress at the surface. The frequencies which result in zero stress are the _eigenfrequencies_.

In [ ]:
# Define a function to compute the stress at the surface
def surface_stress(sol, omega):
    r = sol.t[-1]  # The radius at the surface
    u, v = sol.y[:, -1]  # Displacement and its derivative at the surface
    
    # Compute elastic quantities
    ρ = rho(r)
    L = Lame(r)
    μ = ρ * Vs(r)**2
    λ = L - 2 * μ
    
    # Stress-free condition: radial stress at the surface should be zero
    return L * v + 2 * λ * u / r

# Initial conditions at the center
u_0 = 1.0e-4  # Small displacement at the CMB
v_0 = 0.0     # Zero derivative at the CMB

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

# Let's solve the initial value problem for different values of omega
# Solving for the normal mode means finding the value of omega that
# leads to a stress-free surface condition
omega_values = np.logspace(-2, -1, 20)
for omega in omega_values:
    # Solve the IVP
    sol = solve_ivp(
        radial_ode,
        [dR, R],
        [u_0, v_0],
        args=(omega,),
        t_eval=rvec,
        jac=jac_radial_ode,
        method='LSODA',
    )
    # Compute the surface stress
    stress = surface_stress(sol, omega)
    # Make plots
    line, = ax1.plot(sol.t, sol.y[0, :])
    ax2.plot(omega, stress, 'o', color=line.get_color())

ax1.set_xlabel('Distance from CMB (m)')
ax1.set_ylabel('Radial displacement (m)')
ax2.set_xlabel('Frequency (rad/s)')
ax2.set_ylabel('Surface stress (Pa)')
ax2.grid(True, ls=':', lw=0.6)
plt.tight_layout()

### Using optimization to find the eigenfrequencies and normal modes

We can use `scipy.optimize.minimize` to find the exact eigenfrequencies for given starting guesses. Remember, since there are multiple crossings of the y-axis, we generally need to provide several initial guesses to find each eigenfrequency. The solutions to the ODE for each eigenfrequency will give us the eigenfunctions, $\mathbf{s}$.

In [ ]:
# Let's solve for omega using optimization techniques

# Define the cost function
def cost_function(omega):
    # Solve the IVP
    sol = solve_ivp(
        radial_ode,
        [dR, R],
        [u_0, v_0],
        args=(omega,),
        t_eval=rvec,
        jac=jac_radial_ode,
        method='LSODA',
    )
    # Compute the surface stress
    stress = surface_stress(sol, omega)
    # Return the square of the stress
    return stress**2

# Using the plot above, let's make a list of the rough values of the eigenfrequencies
# with the four lowest values
omega_init_values = [0.01, 0.025, 0.04, 0.05]

fig, ax = plt.subplots()

# Optimize for the eigenfrequency
eigenfrequencies = []
eigenfunctions = []
for omega_guess in omega_init_values:
    res = minimize(cost_function, omega_guess, method='Nelder-Mead')
    omega_eigen = res.x[0]
    print(res.message)
    print('Eigenfrequency:', omega_eigen, '\n')
    
    # Solve the IVP with the eigenfrequency
    # in order to get the eigenfunction (the solution to the ODE)
    sol = solve_ivp(
        radial_ode,
        [dR, R],
        [u_0, v_0],
        args=(omega_eigen,),
        t_eval=rvec,
        jac=jac_radial_ode,
        method='LSODA',
    )
    s = sol.y[0, :] # eigenfunction

    eigenfrequencies.append(omega_eigen)
    eigenfunctions.append(s)

    ax.plot(rvec, s)

ax.set_xlabel('Radius from CMB (m)')
ax.set_ylabel('Eigenfunction (m)')
plt.tight_layout()

## Predicting surface displacement due to density perturbations

In order to compute the predicted surface displacment due to density perturbations, we will use perturbation theory. We integrate the product of a _sensitivity kernel_, $K(r)$, with density variations $\delta \rho(r)$ over depth.

$$
u(R) = \int_0^R K(r', \mathbf{s}) \delta \rho(r') dr'
$$

To perform the integration, we will use `numpy.trapz`. The sensitivity kernel is a function of radius $r$ and a certain eigenfunction $\mathbf{s}$ (which is just one of the normal modes). For this experiment, we will prescribe a density perturbation that is 1% of the reference density, i.e. $\delta \rho(r) = 0.01 \rho(r)$.

In [ ]:
# Define the sensitivity kernel
def sensitivity_kernel(r, s):
    return s / (rho(r) * r)

# Define the density perturbation
def delta_rho(r):
    return 0.01 * rho(r)  # 1% perturbation

fig, ax = plt.subplots()

# Compute the predicted tidal displacement for each eigenfunction
total_delta_u = 0.0
total_K = 0.0
for omega, s in zip(eigenfrequencies, eigenfunctions):
    K = sensitivity_kernel(rvec, s)
    delta_u = np.trapz(K * delta_rho(rvec), rvec)
    total_delta_u += delta_u
    total_K += K
    print(f"Predicted tidal displacement for eigenfrequency {omega:4.3f}: {delta_u:4.3f} meters")
    ax.plot(rvec, K)

ax.plot(rvec, total_K, 'k', lw=3, label='Total sensitivity')
ax.legend(loc='best')
ax.set_xlabel('Radius from CMB (m)')
ax.set_ylabel('Sensitivity')
plt.tight_layout()

print('')
print(f"Final predicted tidal displacement: {total_delta_u:4.3f} meters")

plt.tight_layout()

In [ ]:
plt.plot(rvec, delta_rho(rvec))

## Future work

To summarize, we have created a model that will take as input a profile of density perturbations, $\delta \rho$, and outputs a single value for the predicted tidal displacement at the surface, $u(R)$. This displacement represents the expected deformation of a point on a surface due to density variations beneath that point.

The most complicated part of the forward model is the computation of the normal modes. In practice, __we only have to compute these once__. Once they have been pre-computed, we feed them to the sensitivity kernel function in order to compute the integral for the surface displacement. In this way, we can create a very fast forward model to explore different density variations. Our code for the total inversion workflow would look something like:

In [ ]:
def forward_model(delta_rho, s):

    # Use pre-computed normal modes (eigenfunctions) s
    K = ...

    # Do stuff
    u = np.trapz(...)

    return u

# Load GPS observations of M2 tides
u_gps = np.loadtxt(...)

# Load or predict M2 tides from ocean tidal loading
u_ocean = ...

# Clean the GPS data
u_clean = u_gps - u_ocean - ...

# Run the inversion
estimated_delta_rho = inversion(u_clean, forward_model)